In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

'c:\\Users\\Joaquín Amat\\Documents\\GitHub\\skforecast'

In [2]:
# simulate tiem series
import numpy as np
import pandas as pd
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge

In [3]:
# simulate tiem series
import numpy as np
import pandas as pd

y = np.random.normal(0, 1, 1000)
y = pd.Series(y, index=pd.date_range('2020-01-01', periods=1000, freq='D'), name='users')
exog = pd.DataFrame(np.random.normal(0, 1, (1000, 2)), index=y.index, columns=['exog1', 'exog2'])
exog_test = exog.copy()
exog_test.index = exog.index + len(exog)*exog.index.freq

In [4]:
y_frame = y.to_frame(name=y.name if y.name is not None else 'y')
y_frame

,users
2020-01-01,0.037702
2020-01-02,-0.285581
2020-01-03,-0.443768
2020-01-04,1.565011
2020-01-05,0.361208
...,...
2022-09-22,0.475074
2022-09-23,-0.538478
2022-09-24,1.792705
2022-09-25,-2.337704


In [5]:
from skforecast.utils import transform_dataframe
from skforecast.utils import transform_series
from skforecast.utils import preprocess_y

In [6]:
transformer_y = StandardScaler()
y_transformed = transform_series(
                    series            = y,
                    transformer       = transformer_y,
                    fit               = True,
                    inverse_transform = False
                )
y_values, y_index = preprocess_y(y=y_transformed)

y_inverse = transform_series(
                series            = y_transformed,
                transformer       = transformer_y,
                fit               = False,
                inverse_transform = True
            )

In [7]:
transformer_y_df = StandardScaler()
y_frame_transformed = transform_dataframe(
                df                = y_frame,
                transformer       = transformer_y_df,
                fit               = True,
                inverse_transform = False
            )
y_frame_values, y_frame_index = preprocess_y(y=y_frame_transformed)
y_frame_inverse = transform_dataframe(
                df                = y_frame_transformed,
                transformer       = transformer_y_df,
                fit               = False,
                inverse_transform = True
            )

In [8]:
pd.testing.assert_series_equal(y_transformed, y_frame_transformed[y.name])
pd.testing.assert_index_equal(y_index, y_frame_index)
pd.testing.assert_series_equal(y_inverse, y_frame_inverse[y.name])

In [9]:
forecaster = ForecasterAutoreg(regressor=Ridge(), lags=10, transformer_y=StandardScaler(), transformer_exog=StandardScaler())
forecaster.fit(y=y, exog=exog)
forecaster

ForecasterAutoreg 
Regressor: Ridge() 
Lags: [ 1  2  3  4  5  6  7  8  9 10] 
Transformer for y: StandardScaler() 
Transformer for exog: StandardScaler() 
Window size: 10 
Weight function included: False 
Differentiation order: None 
Exogenous included: True 
Exogenous variables names: ['exog1', 'exog2'] 
Training range: [Timestamp('2020-01-01 00:00:00'), Timestamp('2022-09-26 00:00:00')] 
Training index type: DatetimeIndex 
Training index frequency: D 
Regressor parameters: {'alpha': 1.0, 'copy_X': True, 'fit_intercept': True, 'max_iter': None, 'positive': False, 'random_state': None, 'solver': 'auto', 'tol': 0.0001} 
fit_kwargs: {} 
Creation date: 2024-06-12 19:05:56 
Last fit date: 2024-06-12 19:05:56 
Skforecast version: 0.13.0 
Python version: 3.12.0 
Forecaster id: None 

In [10]:
forecaster.predict(steps=10, exog=exog_test)

2022-09-27   -0.268801
2022-09-28    0.051620
2022-09-29   -0.043229
2022-09-30   -0.064408
2022-10-01   -0.027843
2022-10-02   -0.018004
2022-10-03   -0.027705
2022-10-04   -0.053525
2022-10-05   -0.012990
2022-10-06   -0.021774
Freq: D, Name: pred, dtype: float64

In [11]:
forecaster = ForecasterAutoreg(Ridge(), lags=3)
forecaster.fit(y=pd.Series(np.arange(50), name='y'), exog=pd.Series(np.arange(50, 150, 2), name='exog'))
exog_pred = pd.Series(np.arange(100, 105), index=pd.RangeIndex(start=50, stop=55), name='exog')
predictions = forecaster.predict(steps=5, exog=exog_pred)

In [12]:
from skforecast.utils import input_to_frame
input_to_frame(data=exog_pred, input_name='exog')

,exog
50,100
51,101
52,102
53,103
54,104
